In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

c:\Users\Ashwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import pandas as pd

df = pd.read_csv('path_to_your_dataset.csv')

text_columns = ['Abstract', 'Title', 'Keywords', 'Summary']

for col in text_columns:
    df[col] = df[col].str.lower().str.replace('[^\w\s]', '')

df['CombinedText'] = df[text_columns].fillna('').agg(' '.join, axis=1)


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['CombinedText'].tolist(), show_progress_bar=True)

df['Embedding'] = list(embeddings)


In [ ]:
import faiss
import numpy as np

embeddings_np = np.array(embeddings).astype('float32')

index = faiss.IndexFlatL2(embeddings_np.shape[1])

index.add(embeddings_np)


In [ ]:
def search_entries(query, k=5):
    query_embedding = model.encode([query])

    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)

    results = df.iloc[indices[0]].copy()
    results['Distance'] = distances[0]
    return results

query = "Your search query here"
search_results = search_entries(query)
print(search_results)
